In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import requests
import re
import selenium
import pandas as pd
import html5lib
import math
import lxml
import html.parser
import numpy as np



In [50]:
dict_of_schools={'Princeton University':{'Number of professors':326 , 'School ID':780},
                 'Harvard University':{'Number of professors': 567, 'School ID':399},
                 'Columbia University':{'Number of professors': 893, 'School ID':278},
                 'Massachusetts Institute of Technology':{'Number of professors':346 , 'School ID':580},
                 'Yale University':{'Number of professors':464 , 'School ID':1222},
                 'Stanford University':{'Number of professors':611 , 'School ID':953},
                 'University of Chicago':{'Number of professors':507 , 'School ID':1085},
                 'University of Pennsylvania':{'Number of professors':523 , 'School ID':169},
                 'California Institute of Technology':{'Number of professors':93 , 'School ID':148},
                 'Johns Hopkins University':{'Number of professors':830 , 'School ID':464},
                 
                 'Wichita State University':{'Number of professors':1713 , 'School ID':1197},
                 'Western Kentucky University':{'Number of professors':1669 , 'School ID':1176},
                 'University of Charleston':{'Number of professors':94 , 'School ID':1084},
                 'Regent University':{'Number of professors':485 , 'School ID':4375},
                 'University of Hawaii at Hilo ':{'Number of professors':435 , 'School ID':1105},
                 'Cleveland State University ':{'Number of professors':1862 , 'School ID':244},
                 'Husson University':{'Number of professors':221 , 'School ID':426},
                 'Palm Beach Atlantic University':{'Number of professors':357 , 'School ID':753},
                 'Texas Womans University':{'Number of professors':1311 , 'School ID':1014},
                 'University of Texas at Tyler':{'Number of professors':717 , 'School ID':4171},
                 
                }


In [70]:
#Make list of top and bottom school numbers, to categorize out schools in dataframe.
top = []
bottom = []

for i in range(len(list(dict_of_schools.keys()))):
    if i < 10:
        top.append(dict_of_schools[list(dict_of_schools.keys())[i]]['School ID'])
    else:
        bottom.append(dict_of_schools[list(dict_of_schools.keys())[i]]['School ID'])



[780, 399, 278, 580, 1222, 953, 1085, 169, 148, 464] [1197, 1176, 1084, 4375, 1105, 244, 426, 753, 1014, 4171]


The keys for each element in list_of_professor_dicts are:,
'tDept' = Department (eg. science, spanish, law ect),
'tSid' = The school id (eg. 1222 for all Yale professors),
'institution_name' = Name of university,
'tFname' = First name,
'tMiddlename' = Middle name,
'tLname' = Last name,
'tid' = Teacher id,
'tNumRatings' = Number of ratings,
'rating_class' = How is their score (eg. good, average, poor, zero),
'contentType' = Have only seen this to return 'TEACHER',
'categoryType'= Have only seen this to return 'PROFESSOR',
'overall_rating' = Average rating (float)

In [528]:
def get_all(school_id, number_of_professors): #BRUG DEN HER OG KUN DEN HER
    """This function takes as an argument, a school ID(integer) and the number of professors(integer) at the given school.
        reads the correct URL's ans saves the parsed data into csv files with name corresponding with the school ID.
    """
    number_of_pages=math.ceil(number_of_professors/20)
    list_of_dfs=[]
    for j in range(1,number_of_pages+1): #all page numbers
        url=f"https://www.ratemyprofessors.com/filter/professor/?&page={j}&filter=teacherlastname_sort_s+asc&query=*%3A*&queryoption=TEACHER&queryBy=schoolId&sid="+str(school_id)
        response = requests.get(url) #this response is a dict of dicts.
        list_of_dicts=response.json() #keys are: ['professors', 'searchResultsTotal', 'remaining', 'type']
        list_of_professor_dicts=list_of_dicts['professors'] #enter the dict with info regarding professors
        list_of_dfs.append(pd.concat([pd.DataFrame(list_of_dicts['professors'] [i], index=[i]) for i in range(len(list_of_professor_dicts))],
                  ignore_index=True)) #appends a dataframe with information of all professors on the given page to list
    df=pd.concat([list_of_dfs[i] for i in range(len(list_of_dfs))], axis= 0).reset_index(drop=True) #after going through all pages. Concatenate all df's into one
    list_of_teacher_ids=list(df.tid) #create a list of all professor id's for the given university
    list_of_comments_for_each_teacher=[]
    list_of_tags_for_each_teacher=[]
    for i in range(len(list_of_teacher_ids)):
        my_url = f'https://www.ratemyprofessors.com/ShowRatings.jsp?tid={list_of_teacher_ids[i]}'#enter URL for each professor
        my_response = requests.get(my_url)
        html = my_response.text
        soup = BeautifulSoup(html,'lxml')
        my_text = soup.findAll('div', {'class':'Comments__StyledComments-dzzyvm-0 gRjWel'})#list of comments unparsed
        my_tags = soup.findAll('span', {'class':'Tag-bs9vf4-0 hHOVKF'})#list of comments unparsed
        finished_comments=[i.text for i in my_text]
        finished_tags=[i.text for i in my_tags]
        list_of_comments_for_each_teacher.append(finished_comments)
        list_of_tags_for_each_teacher.append(finished_tags)
    df['Comments']=list_of_comments_for_each_teacher #add a column to dataframe where each cell contains a list of all comments for the given professor.
    df['Tags']=list_of_tags_for_each_teacher#add a column to dataframe where each cell contains a list of all tags for the given professor.
    df.to_csv(f'{school_id}.csv') #saves the dataframe as a csv file
    
    ##Second dataframe
    list_of_comments_and_ids=[]
    for i in range(len(list_of_teacher_ids)):
        my_url = f'https://www.ratemyprofessors.com/ShowRatings.jsp?tid={list_of_teacher_ids[i]}'
        my_response = requests.get(my_url, headers={'user-agent': 'For an exam at the university of copenhagen for the course ISDS'})
        html = my_response.text
        soup = BeautifulSoup(html,'lxml')
        my_comments = soup.findAll('div', {'class':'Comments__StyledComments-dzzyvm-0 gRjWel'})#list of tags unparsed
        my_date = soup.findAll('div', {'class':'TimeStamp__StyledTimeStamp-sc-9q2r30-0 bXQmMr RatingHeader__RatingTimeStamp-sc-1dlkqw1-3 BlaCV'})#list of dates unparsed
        my_quality = soup.findAll('div', {'class':'CardNumRating__StyledCardNumRating-sc-17t4b9u-0 eWZmyX'})#list of dates unparsed
        finished_quality=[i.text for i in my_quality]
        finished_comments=[i.text for i in my_comments]
        finished_dates=[i.text for i in my_date]
        if len(finished_comments)!=0:
            list_of_comments_and_ids.append([finished_comments, [k[-3:] for k in finished_quality[0::2]],[k[-3:] for k in finished_quality[1::2]], finished_dates[0::2],[list_of_teacher_ids[i] for j in range(len(finished_comments))]])
    hej=pd.concat([pd.DataFrame(list_of_comments_and_ids[i]).T for i in range(len(list_of_comments_and_ids))]) #creates the dataframe
    hej.columns=['Comments', 'Quality','Difficulty','Dates', 'TeacherID']
    hej=hej.reset_index(drop=True)
    hej.to_csv(f'{school_id}comments.csv')
    return 


In [520]:
#get_all(schoolID, number_of_prof)

In [3]:
hej1=pd.read_csv('417.csv')
hej2=pd.read_csv('4171comments.csv')
hej2

,Unnamed: 0,Comments,Quality,Difficulty,Dates,TeacherID
0,0,"He clearly loves what he does, and his passion...",4.0,4.0,"Jul 13th, 2021",2653027
1,1,"He clearly loves what he does, and his passion...",4.0,4.0,"Jul 7th, 2021",2653027
2,2,"A little strict, but overall a great prof. I w...",5.0,4.0,"Apr 28th, 2021",2653027
3,3,Dr. Adbelal is tough but I love him! This was ...,5.0,4.0,"Dec 13th, 2020",2653027
4,4,Dr. Abdelal is one of the best professors Ive ...,NaN,NaN,NaN,2653027
...,...,...,...,...,...,...
5071,5071,Difficult to understand and not very helpful. ...,1.5,3.0,"Dec 23rd, 2009",608528
5072,5072,MUY MAL!! This teacher is unorganized and scat...,1.0,1.0,"Dec 22nd, 2005",608528
5073,5073,very sweet disposition -- very willing to help...,5.0,1.0,"Dec 20th, 2005",608528
5074,5074,"This professor is very helpful, wants her stud...",5.0,1.0,"Sep 6th, 2005",608528


In [529]:
#For running the get_all function for all universities. Be ware that running this takes several hours.
"""
for i in list(dict_of_schools.keys()):
    print(i)
    school_to_run=i
    number_of_prof=dict_of_schools[school_to_run]['Number of professors']
    schoolID=dict_of_schools[school_to_run]['School ID']
    get_all(schoolID, number_of_prof)
"""

"\nfor i in list(dict_of_schools.keys()):\n    print(i)\n    school_to_run=i\n    number_of_prof=dict_of_schools[school_to_run]['Number of professors']\n    schoolID=dict_of_schools[school_to_run]['School ID']\n    get_all(schoolID, number_of_prof)\n"

In [11]:
#dict_of_schools[]['School ID']
dict_of_schools

{'Princeton University': {'Number of professors': 326, 'School ID': 780},
 'Harvard University': {'Number of professors': 567, 'School ID': 399},
 'Columbia University': {'Number of professors': 893, 'School ID': 278},
 'Massachusetts Institute of Technology': {'Number of professors': 346,
  'School ID': 580},
 'Yale University': {'Number of professors': 464, 'School ID': 1222},
 'Stanford University': {'Number of professors': 611, 'School ID': 953},
 'University of Chicago': {'Number of professors': 507, 'School ID': 1085},
 'University of Pennsylvania': {'Number of professors': 523, 'School ID': 169},
 'California Institute of Technology': {'Number of professors': 93,
  'School ID': 148},
 'Johns Hopkins University': {'Number of professors': 830, 'School ID': 464},
 'Wichita State University': {'Number of professors': 1713, 'School ID': 1197},
 'Western Kentucky University': {'Number of professors': 1669,
  'School ID': 1176},
 'University of Charleston': {'Number of professors': 94,

In [17]:
all_df=[]
for i in list(dict_of_schools.keys()):
    nr = dict_of_schools[i]['School ID']
    df = pd.read_csv(f'{nr}.csv')
    df['SchoolID'] = nr
    all_df.append(df)
    
df_all = pd.concat(all_df)
df_all.to_csv('concatinated_df')
    
    

In [18]:
all_df=[]
for i in list(dict_of_schools.keys()):
    nr = dict_of_schools[i]['School ID']
    df = pd.read_csv(f'{nr}comments.csv')
    df['SchoolID'] = nr
    all_df.append(df)
    
df_all = pd.concat(all_df)
df_all.to_csv('concatinated_comments_df')
    

In [6]:
hej1=pd.read_csv('concatinated_df')
hej2=pd.read_csv('concatinated_comments_df')
hej1


,Unnamed: 0,Unnamed: 0.1,tDept,tSid,institution_name,tFname,tMiddlename,tLname,tid,tNumRatings,rating_class,contentType,categoryType,overall_rating,Comments,Tags,SchoolID
0,0,0,Philosophy,780,Princeton University,Uriel,NaN,Abulof,2608286,0,zero,TEACHER,PROFESSOR,NaN,[],[],780
1,1,1,Art History,780,Princeton University,Al,NaN,Acres,336888,11,good,TEACHER,PROFESSOR,4.7,"['Professor Acres is incredible--friendly, kno...","['Amazing lectures', 'Gives good feedback', 'I...",780
2,2,2,Economics,780,Princeton University,Alicia,NaN,Adsera,1695875,3,poor,TEACHER,PROFESSOR,1.8,"['Extremely boring lectures, poor teaching.', ...",[],780
3,3,3,Engineering,780,Princeton University,Ilhan,NaN,Aksay,2020549,1,good,TEACHER,PROFESSOR,5.0,"[""Professor Aksay was undoubtedly the best pro...",[],780
4,4,4,Architecture,780,Princeton University,Amale,NaN,Andros,1348710,1,poor,TEACHER,PROFESSOR,1.0,[''],[],780
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14020,712,712,Criminal Justice,4171,University of Texas at Tyler,Shudong,NaN,Zhang,1859429,0,zero,TEACHER,PROFESSOR,NaN,[],[],4171
14021,713,713,Accounting,4171,University of Texas at Tyler,Ke,NaN,Zhong,590038,15,average,TEACHER,PROFESSOR,2.6,"[""great teacher. she prepares you for the CPA ...",[],4171
14022,714,714,Psychology,4171,University of Texas at Tyler,Kirk,NaN,Zinck,2110503,0,zero,TEACHER,PROFESSOR,NaN,[],[],4171
14023,715,715,Education,4171,University of Texas at Tyler,Staci,NaN,Zolkolski,2206126,3,good,TEACHER,PROFESSOR,5.0,['She was a great professor that really cared ...,"['Gives good feedback', 'Tough grader', 'Get r...",4171


In [79]:
#Fjern contenttype og categorytype
df = hej1.drop(['contentType', 'categoryType', 'Unnamed: 0.1', 'Unnamed: 0' ], axis =1)

df['status'] = np.where(df['tSid'].isin(top), 'top', 'bottom')

df.to_csv('df_final.csv')
df_no_comment = df[df['Comments'] == '[]']    



###Rens anden og lav til csv
df_comments = hej2.loc[~hej2['TeacherID'].isin(list(df_no_comment['tid']))]\
         .drop(['Unnamed: 0.1', 'Unnamed: 0' ], axis =1)

df_comments['Dates'] = pd.to_datetime(df_comments.Dates)
df_comments['status'] = np.where(df_comments['SchoolID'].isin(top), 'top', 'bottom')
df_comments = df_comments[df_comments['Comments'] != 'No Comments']
df_comments = df_comments['Comments'].dropna()


df_comments.to_csv('df_comments_final.csv')

In [80]:
df_comments

,Comments,Quality,Difficulty,Dates,TeacherID,SchoolID,status
0,"Professor Acres is incredible--friendly, knowl...",5.0,4.0,2017-05-02,336888,780,top
1,He is an amazing professor- I definitely recom...,5.0,3.0,2017-04-28,336888,780,top
2,Great!,3.0,3.0,2012-03-23,336888,780,top
3,Awesome,5.0,1.0,2011-02-01,336888,780,top
4,Al is an absolutely great professor. His semin...,5.0,4.0,2007-10-21,336888,780,top
...,...,...,...,...,...,...,...
81712,Difficult to understand and not very helpful. ...,1.5,3.0,2009-12-23,608528,4171,bottom
81713,MUY MAL!! This teacher is unorganized and scat...,1.0,1.0,2005-12-22,608528,4171,bottom
81714,very sweet disposition -- very willing to help...,5.0,1.0,2005-12-20,608528,4171,bottom
81715,"This professor is very helpful, wants her stud...",5.0,1.0,2005-09-06,608528,4171,bottom
